# __Medical Unlearning__

### __Examine Data__

We consider several medical datasets from MedMNIST (https://medmnist.com/) shown in the table bellow:

<center><img src="pics/data_table.png"/></center>

Lets have a look at the BloodMNIST dataset. To examine a dataset use the following command line:

*__python examine_dataset.py --dataset {dataset_name}__*

Here is the command for BloodMNIST:

*__python examine_dataset.py --dataset BloodMNIST__*

This script will tell you how much of each class exists in the dataset, as well as the dataset split, and a grid of the data samples (with the associated class):

<center><img src="pics/blood_data.png" width="500" /></center>

### __Train Model__

To train a model on one of these datasets we use the following command line:

*__python main_train.py  --save_model_id {name}  --dataset {dataset_name}__*

The save_model_id is a unique identifier for the trained model and can be any string. More arguments can be specified here, such as epochs, learning rate, and random seed, all available arguments can be seen in the arg_parser.py script. Bellow is an example:

*__python main_train.py  --save_model_id {name}  --dataset {dataset_name}  --seed {seed_number} --epochs {epochs} --lr {learning_rate} --batch_size {batch_size}__*

For example, we will run the following command:

*__python main_train.py  --save_model_id example_blood_model  --dataset BloodMNIST  --seed 1 --epochs 50 --lr 0.01 --batch_size 50__*

The script will save both the last and best checkpoints to the directory:
**model_weights/{name}_best.pt** or **model_weights/{name}_last.pt**




### __Test Model__

Lets quickly test the performance of this model using the following command line:

*__python main_test.py --model_id {name} --dataset {dataset_name}__*

Here is the full command line we will use (remember the _best or _last for model_id), also be mindful that batch_size and num_workers are optimal for your computer:

*__python main_test.py --model_id example_blood_model_best  --dataset BloodMNIST  --batch_size 50  --num_workers 0__*

Bellow is the output:

<center><img src="pics/blood_base_model_test.png" width="500" /></center>

Using the number keys, you can sample data from specific classes, the script will also output the class wise performance of the model to the terminal:

<center><img src="pics/blood_test_class.png" width="500" /></center>

### __Create Unlearned Model__

#### __Generate Masks__

First we must create the saliency map for the base model, in our case, the base model we will use is __example_blood_model_best__

To create the saliency map, simply run this command:

*__python generate_mask.py --model_id {model_name} --dataset {dataset_name}__*

So our command will be:

*__python generate_mask.py --model_id example_blood_model_best --dataset BloodMNIST__*

This will create a directory in SaliencyMaps/{model_name}/ which will contain all the masks, different threshholds will result in different performance, so some work should be done to decide which mask is the most effective. For the purpose of this notebook, lets just use threshhold 0.5

#### __Forget Class__

Lets attempt to forget class 3. To do this, simply use the following command line:

*__python main_unlearn.py --model_id {model_name} --dataset {dataset_name}   --unlearn {unlearn_method_id}  --class_to_forget {class_number}  --mask_thresh {threshhold}__*

You can specify more paramentres:

*__python main_unlearn.py --model_id {model_name} --dataset {dataset_name}   --unlearn {unlearn_method_id}  --class_to_forget {class_number}  --mask_thresh {threshhold} --unlearn_epochs {epochs}    --unlearn_lr {learning_rate}  --unlearn_batch_size {batch_size}__*

Bellow is a table of unlearning techniques and the associated unlearn_method_id:

<center><img src="pics/unlearning_methods.png" width="500" /></center>

Note that different techniques will require different ideal values for learing rate and epochs etc, therefore some testing is required to find the ideal parametres.

Bellow is the full command line we will use to create an unlearned model from our base model __example_blood_model_best__ that has forgotten class 3, lets use *Random Labelling with SalUn*:

*__python main_unlearn.py --model_id example_blood_model_best --dataset BloodMNIST   --unlearn RL_with_SalUn     --class_to_forget 3 --mask_thresh 0.5 --unlearn_epochs 5    --unlearn_lr 0.001  --unlearn_batch_size 100__*

Note some unlearning methods don't need a mask threshhold, so it will ignore the --mask_thresh argument.
This will create a model that will have the unique ID:

__example_blood_model_best_FORGOTTEN_3_RL_WITH_SALUN_mask5_unlearnEpochs5_unlearnlr001__

### __Test and Compare Unlearned Model__

To check how well our unlearned model has done, we can run the following command line:

*__python main_test_unlearn.py  --model_id {model_name}  --dataset {dataset_name}  --class_to_forget {class_number}  --batch_size {batch_size}__*

In our case, we will run the following command:

*__python main_test_unlearn.py  --model_id example_blood_model_best_FORGOTTEN_3_RL_WITH_SALUN_mask5_unlearnEpochs5_unlearnlr001  --dataset BloodMNIST  --class_to_forget 3  --batch_size 100__*

This will create a dataset without the class 3, and test how well the model does on it. The script also creates a dataset only containing class 3 to check how well we have forgotten. The three criterea we are concerned about are:

- __Unlearn Accuracy__ (How well we have forgotten)
- __Remaining Accuracy__ (How well the model performs on data it has seen before) 
- __Test Accuracy__ (How well the model performs on data it hasn't seen before)


<center><img src="pics/unlearn_test_output.png" width="500" /></center>

Unlearn accuracy is not amazing, but this is without parametre optimisation, so lets continue. We can also run this script on the base model, to get an idea of how the original model performs on the forgotten dataset:

*__python main_test_unlearn.py     --model_id example_blood_model_best  --dataset BloodMNIST  --class_to_forget 3 --batch_size 100__*

<center><img src="pics/base_model_test.png" width="500" /></center>

Great, so we have partially forgotten class 3, and our model accuracy has not degraded. To examine the model's performance, we can run the main_test.py script again for our unlearned model:

*__python main_test.py --model_id example_blood_model_best_FORGOTTEN_3_RL_WITH_SALUN_mask5_unlearnEpochs5_unlearnlr001    --dataset BloodMNIST    --batch_size 50 --num_workers 0__*



<center><img src="pics/test_unlearn_main.png" width="700" /></center>

Lets have a look at class 3:

<center><img src="pics/test_unlearn_main3.png" width="700" /></center>

<center><img src="pics/test_unlearn_main_classwise.png" width="700" /></center>